# P3 Automatic test!

## 使用方法详见本文末

定义路径，导入相关库

In [17]:
import re
import random
import os
from time import sleep
import subprocess

MARS_PATH = r"C:\Users\Harudog\Desktop\Use\Working Area\Tools\Co\Mars4_5.jar"
CPU      = r"C:\Users\Harudog\Desktop\Use\Working Area\Src Code\coCode\2025\coHw\p3\cpu_single.circ"
PARTNER  = r"C:\Users\Harudog\Desktop\Use\Working Area\Src Code\coCode\2025\coHw\p3\cpu_single.circ"
LOGISIM  = r"C:\Users\Harudog\Desktop\Use\Working Area\Tools\Co\logisim-win-2.7.1.exe"
TXT      = "command.txt"
RESULT   = "CPU_RESULT.txt"
RESULT_PARTNER   = "CPU_RESULT_P.txt"

spinner_running = False

生成随机汇编指令

In [18]:
"""
    Generate Assemble with:
    ori, lui,
    add, sub, lw, sw,
    beq
    And dump them into Hexadecimal txt file.
"""

# Reg Index, Just for better look
regs = [f"{i}" for i in range(1, 32)]

R_type = ['add', 'sub']
I_type = ['ori', 'lui', 'lw', 'sw']
J_type = ['j', 'jal', 'jr']

def r_type(rs, rt, rd):
    op = random.choice(R_type)
    return f"{op} ${rd}, ${rs}, ${rt}"

def i_type(rs, rt, imm):
    op = random.choice(I_type)
    if op == 'beq':
        return f"{op} ${rs}, ${rt}, {imm}"
    elif op in ['lw', 'sw']:
        return f"{op} ${rt}, {imm}(${rs})"  # lw $rt, offset($rs)
    else:
        return f"{op} ${rt}, {imm}"

# def J(imm, random_code):
# no more j instruction for hw...

def generate_random_code(num_instructions=10):
    """
        Generate random code(I don't think 10 ins is enough but just use it?)
    """
    code = []
    for _ in range(num_instructions):
        if random.random() < 0.4:
            rs, rt, rd = random.sample(regs, 3)
            code.append(r_type(rs, rt, rd))
        else:
            rs = random.choice(regs)
            rt = random.choice(regs)
            if random.choice(I_type) == 'beq':
                imm = str(random.randint(-128, 127) * 4)
            else:
                imm = str(random.randint(0, 65535)) if random.random() < 0.8 else hex(random.randint(0, 65535))
            code.append(i_type(rs, rt, imm))
    return "\n".join(code)


后续写入辅助函数

In [19]:
def write_to_file(filename="AssemblyP3.asm"):
    # print("Generating Assembly!")
    code = generate_random_code(100)
    with open(filename, 'w') as f:
        sleep(0.5)
        print("File Open!")
        f.write(code)
        sleep(0.5)
    # print(f"Code Generated in {filename} Successfully!")

def inject(circ, code):
    txt = open(circ, encoding="utf-8").read()
    txt = re.sub(r'(addr/data: 10 32\n).*?(</a>)',
                 r'\1' + code + r'\n\2',
                 txt, count=1, flags=re.S)
    open(circ, "w", encoding="utf-8").write(txt)

# def spinner(spinner_running):
#     """Waiting and Spinning"""
#     spin_chars = ['\\', '|', '/', '-']
#     i = 0
#     while spinner_running:
#         sys.stdout.write(f'\rRunning... {spin_chars[i]}')
#         sys.stdout.flush()
#         i = (i + 1) % len(spin_chars)
#         time.sleep(0.1)
#     sys.stdout.write('\r' + ' ' * 20 + '\r')
#     sys.stdout.flush()

def cmd_running(result):
    if result == 0 and os.path.exists("command.txt"):
        print("Well Done！command.txt Generated")
        with open("command.txt", 'r') as f:
            content = f.read().strip()
        with open("command.txt", 'w') as f:
            f.write("v2.0 raw\n")
            f.write(content + "\n")
    else:
        print("WTF, Errors happened!!!")

生成机器码

In [20]:
def auto_mars_code():
    filename="AssemblyP3.asm"

    # warning_check = input("Attention! Please check your PATH!!! Input OK to continue:")
    # if warning_check.lower() == "ok":
    #     print("OK! Your PATH could be right...")
    # else:
    #     print("Error! Your PATH could be wrong...")
    # write_to_file(filename)

    cmd = (
        f'java -jar "{MARS_PATH}" '
        f'"{filename}" '
        f'p nc mc CompactDataAtZero a dump .text HexText command.txt'
    )

    # print("Running MARS...")
    # print(cmd)
    sleep(0.5)
    # print("-" * 60)

    result = os.system(cmd)
    cmd_running(result)

开始生产`log`文件

In [21]:
def auto_log():
    code = open(TXT, encoding="utf-8").read().strip()

    inject(CPU, code)
    inject(PARTNER, code)

    process = None

    confirm = "y"
    if confirm.strip().lower() == "y" or confirm.strip().lower() == "yes":
        try:
            # print("Running Logisim ...")

            process = subprocess.Popen([
                "java", "-jar", LOGISIM,
                CPU, "-tty", "table"
            ], stdout=open(RESULT, "w", encoding="utf-8"))

            process.wait()

            process = None

            # print(f"\nWell Done! Finished writing {RESULT}")

            process = subprocess.Popen([
                "java", "-jar", LOGISIM,
                PARTNER, "-tty", "table"
            ], stdout=open(RESULT_PARTNER, "w", encoding="utf-8"))

            process.wait()


            # print(f"\nWell Done! Finished writing {RESULT_PARTNER}")

        except KeyboardInterrupt:
            print("\nInterrupting...")
            if process:
                process.terminate()
                try:
                    process.wait(timeout=5)
                except subprocess.TimeoutExpired:
                    process.kill()
            print("Ok, done interrupted")
    else:
        print("ok, we got it.")

比较`log`文件是否相同

In [22]:
import filecmp
def simple_cmp(file1, file2):
    if filecmp.cmp(file1, file2, shallow=False):
        # print(True)
        return True
    else:
        # print(False)
        return False
# simple_cmp(RESULT, RESULT_PARTNER)

Final test:

In [25]:
def final_function():
    auto_mars_code()
    auto_log()
    return simple_cmp(RESULT, RESULT_PARTNER)

def happy_test(num = 10):
    for i in range(num):
        try:
            print(f"Test {i}:")
            print("Start!".center(20, "-"))
            auto_mars_code()
            if final_function():
                print("Accepted:)")
            else:
                print("Wrong Answer:(")
            print("End!".center(20, "-"))
        except KeyboardInterrupt:
            print("\nInterrupting...")
            print("Finished".center(20, "-"))
            print("You stopped the program, but why?")
            break

check_path = input("The path should be absolute path of your circ code, mars and logisim.\n You sure you checked them?(input y/n)")
if check_path.lower() == "y" or check_path.lower() == "yes":
    num = int(input("How many tests?"))
    while num <= 0 or num > 100:
        num = int(input("Not a good number, How many test?"))
    happy_test(num)
else:
    print("ok, we got it, please wait...")
    for i in range(10):
        sleep(0.5)
        print(".", end='')
    print("Please input your absolute path of your circ code, mars and logisim:")
    MARS_PATH   = input("Input your MARS path.")
    CPU         = input("Input your CPU circ path.")
    PARTNER     = input("Input your PARTNER circ path.")
    LOGISIM     = input("Input your LOGISIM circ path.")
    try:
        num = int(input("How many test?"))
        while num <= 0 or num > 100:
            num = int(input("Not a good number, How many test?"))
        happy_test(num)
    except:
        print("Something went wrong. Maybe you input a wrong path? Joker.")



Test 0:
-------Start!-------
Well Done！command.txt Generated
Well Done！command.txt Generated
Accepted:)
--------End!--------
Test 1:
-------Start!-------
Well Done！command.txt Generated
Well Done！command.txt Generated
Accepted:)
--------End!--------


## 使用方法

全心全意为人民服务，添加了提示框，您可以一键运行全部`cell`，然后提示框会询问您的`logisim`等组件的绝对地址。

然后输入运行样例数，会生成随机`n`个样例进行判断。

实际上，我应该把默认地址改在本文件夹下，但是我太懒了，就懒得改配置文件了。

似乎还有bug但是我太懒啦

运行环境：
    - IPython          : 9.5.0
    - ipykernel        : 6.30.1
    - ipywidgets       : not installed
    - jupyter_client   : 8.6.3
    - jupyter_core     : 5.8.1
    - jupyter_server   : 2.17.0
    - jupyterlab       : 4.4.9
    - nbclient         : 0.10.2
    - nbconvert        : 7.16.6
    - nbformat         : 5.10.4
    - notebook         : not installed
    - qtconsole        : not installed
    - traitlets        : 5.14.3
所有的库函数都没什么特别的声明

## Reference
- [单周期CPU自动化检测](https://foreveryolo.top/posts/6588/index.html)
- [Command-line Verification:Logisim Tutorial](https://foreveryolo.top/posts/6588/index.html)